In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from tensorflow import keras
import shap
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

In [50]:
%load_ext cudf.pandas
import pandas as pd

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [3]:
%%time
# Set random seed for reproducibility
np.random.seed(42)

# Function to read large CSV files in chunks
def read_csv_in_chunks(file_path, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunks.append(chunk)
    return pd.concat(chunks)

CPU times: user 51 µs, sys: 16 µs, total: 67 µs
Wall time: 71 µs


In [4]:
%%time
train_data = read_csv_in_chunks('/kaggle/input/amex-default-prediction/train_data.csv')

CPU times: user 2min 57s, sys: 20.3 s, total: 3min 17s
Wall time: 5min 21s


In [5]:
# Read labels first
print("Reading train_labels.csv...")
train_labels = read_csv_in_chunks('/kaggle/input/amex-default-prediction/train_labels.csv')

Reading train_labels.csv...


In [6]:
%%time
# Sample 20% of observations randomly
print("Sampling 20% of observations...")
train_labels_sample = train_labels.sample(frac=0.2, random_state=42)
    
# Merge with train_data to create the development sample
train_data_sample = train_data.merge(train_labels_sample, on="customer_ID")

# Save the development sample for future use
train_data_sample.to_csv("train_data_sample.csv", index=False)

# Display the new shape and first few rows
print("Development Sample Shape:", train_data_sample.shape)
print(train_data_sample.head())

Sampling 20% of observations...
Development Sample Shape: (1107082, 191)
                                         customer_ID         S_2       P_2  \
0  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-03-11  0.374606   
1  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-04-22  0.414269   
2  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-05-12  0.413310   
3  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-06-10  0.328983   
4  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-07-19  0.496989   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.033519  0.044293  1.008622  0.001470  0.459235  0.002339  0.006168  ...   
1  0.002516  0.059667  0.123964  0.004374  0.434148  0.001405  0.052130  ...   
2  0.003285  0.053418  0.304955  0.002316  0.415906  0.009388  0.048780  ...   
3  0.038574  0.049463  0.115654  0.004654  0.416112  0.003223  0.081001  ...   
4  0.005552  0.041452  0.133631  0.007363  0.419864  0.003